In [ ]:
!pip install --upgrade pip

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version
# Install pyspark
! pip install --ignore-installed pyspark==2.4.5
# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.7.0

In [ ]:
import sparknlp
spark = sparknlp.start()
sparknlp.version()
spark.version

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.ml.feature import *
from pyspark.ml.classification import *

In [ ]:
spark_df = spark.read.option("header", "true").option("multiLine", "true").option("quote", "\"").option("escape", "\"").option("inferSchema", "true").csv('../input/suicide-watch/Cleaned_Depression_Vs_Suicide.csv', sep = ',')

In [ ]:
spark_df.show()

In [ ]:
spark_split_df = spark_df

(train_data, test_data, val_data) = spark_split_df.randomSplit([0.1, 0.01, 0.8], 24)

print("Train length", train_data.count())
print("Test length", test_data.count())
print("validation length", val_data.count())

In [ ]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("class")\
  .setMaxEpochs(20)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

In [ ]:
pipelineModel = pipeline.fit(train_data)

In [ ]:
preds = pipelineModel.transform(test_data)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
df = preds.select('class', 'text', "prediction.result").toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

print(accuracy_score(df['class'], df.result))